In [3]:
import pandas as pd
import numpy as np

# Loading Raw Data

In [44]:
# os.path.join(“..“, “raw_data”, “companies.csv”)
comps = pd.read_csv('..\\raw_data\\companies.csv')
acquisitions = pd.read_csv('..\\raw_data\\acquisitions.csv')
degrees = pd.read_csv('..\\raw_data\\degrees.csv')
founders = pd.read_csv('..\\raw_data\\founders.csv')
rounds = pd.read_csv('..\\raw_data\\funding-rounds.csv')
funds = pd.read_csv('..\\raw_data\\funds.csv')
investments = pd.read_csv('..\\raw_data\\investments.csv')
ipos = pd.read_csv('..\\raw_data\\ipos.csv')
milestones = pd.read_csv('..\\raw_data\\milestones.csv')
people = pd.read_csv('..\\raw_data\\people.csv')
relationships =pd.read_csv('..\\raw_data\\relationships.csv')

# Feature Engineering

#### Change to use in all the functions

In [3]:
comps = comps.rename(columns={'id': 'object_id'})

## 2  1 - Time between founding date and specific investment (Diff_time_founded_invest)

changes are made !!!!

In [10]:
def diff_foundation_fundround(comps_df, rounds_df):
    
    '''Returns a merge table between companies and funding rounds. One column added: difference between the date of the companies fundations
    and each round of investment , in MONTHS. It includes companies_id repeated for each round that is realated with the company '''
    
    comps = comps_df
    rounds = rounds_df
    
    comps = comps.rename(columns={'id': 'object_id'})
    comps_rounds = comps.merge(rounds, how = 'left', on = 'object_id')
    comps_rounds.founded_at = pd.to_datetime(comps_rounds.founded_at)
    comps_rounds.funded_at = pd.to_datetime(comps_rounds.funded_at)
    comps_rounds['Diff_time_founded_invest'] = abs(comps_rounds.funded_at - comps_rounds.founded_at)/np.timedelta64(12, 'M')
    
    return comps_rounds

## 3  3 - Time between Series A and specific investment

#changes are made !!!!!!!!!

In [11]:
def time_serie_investments(rounds_data, reference):

    ''' company id partitioned. Take a serie X as reference: values = round number wished. And create a new column 
    'data_serie_reference' which contains the min date of this serie for each company.
    And also create a new column 'time_serie_investment' with the difference between each round and the date of reference serie, in MONTHS'''
    
    rounds.funded_at = pd.to_datetime(rounds.funded_at)
    
    min_dates = {}
    for id_company in rounds[rounds.number_of_round == reference].object_id.unique():
        #list_dates_by_comp: list of each company which has round number wished
        list_dates_by_comp = rounds[rounds.number_of_round == reference].funded_at[rounds.object_id == id_company]
        min_dates[id_company] = min(list_dates_by_comp)

    rounds['date_serie_reference'] = rounds.object_id.map(min_dates)
    rounds['time_serie_investment'] =  (rounds.funded_at - rounds.date_serie_reference)/np.timedelta64(12, 'M')
    
    return rounds

## Get_round: Get the rounds filtered by one position (of round)

In [14]:
def get_round(rounds, n_reference):
    ''' filter the rounds according the position, ej: 1 round for each company, 2 round for each company
    '''
    
    rounds ['number_of_round'] = rounds.groupby('object_id').cumcount() + 1
    return rounds [rounds.number_of_round == n_reference]

## [Dependent on 1] 2 - Time to series A round 

In [12]:
def diff_fund_found_filter(comps, rounds, serie_filtered):
    ''' filter the table from diff_foundation_foundround (merge comps and rounds), by a particular serie (values = a, b ,c ,angel, venture?)'''
    get_table = diff_foundation_fundround(comps, rounds)
    
    return get_table[get_table.funding_round_code == serie_filtered]

## [Dependent on 3] 4 - Number of rounds before Series A

In [13]:
def num_rounds_before(comps, rounds, serie_reference):
    ''' return the companies table with a new column num_round_before which is the number of rounds before the serie you want. time_serie_investments applied inside.'''

    comps = comps.rename(columns={'id': 'object_id'})
    get_table = time_serie_investments(rounds, serie_reference)
    get_table = get_table[get_table.time_serie_investment<0]
    get_table = get_table.groupby('object_id').count()
    get_table = get_table['id'].reset_index(name="num_round_before")
    merge = comps.merge(get_table, how ='left', on = 'object_id')
    return merge

## 7 - Time between Series A and IPO

In [12]:
#to be merged to companies table
def ipos_by_round(rounds, ipos, comps, n_reference):
    ''' get your IPOss matched with a number of round merged on companies'''
    
    ipos = ipos.drop_duplicates(subset = 'object_id')
    comps = comps.rename(columns={'id': 'object_id'})
    
    get_rounds = get_round(rounds, n_reference)
    merge_1 = ipos.merge(get_rounds, how = 'left', on = 'object_id')
    
    merge_2 = merge_1.merge(comps, how = 'left', on = 'object_id')
    
    comps = comps.rename(columns={'object_id': 'id'})
    
    return merge_2

## Acquisitions_by_round:  acquisition table with a number of orund merged on companies

In [165]:
def acquisitions_by_round(rounds, acquisitions, comps, n_reference):
    
        '''get your matched acquisition table with a number of orund merged on companies'''
        
        comps = comps.rename(columns={'id': 'object_id'})        
        acquisitions = acquisitions.rename(columns={'acquired_object_id': 'object_id'})
    
    
        get_rounds = get_round(rounds, n_reference)
        merge = acquisitions.merge(get_rounds, how = 'left', on = 'object_id')
        
        merge_2 = merge.merge(comps, how = 'left', on = 'object_id')
        
        
        return merge.drop(columns= ['created_by', 'created_at', 'updated_at', 'source_url_x', 'source_url_y' ])

## Merge the Number of Founders By Company

In [56]:
def number_of_fuounders(comps, relationships):
    ''' get the number of founders for each company in the company table'''
    relationship_founders = relationships[relationships.founder==True]
    founders_by_comp = pd.DataFrame(relationships_founders.groupby('relationship_object_id')['relationship_object_id'].count())
    founders_by_comp = founders_by_comp.rename(columns = {'relationship_object_id' : 'number_of_founders'}).reset_index().rename(columns ={'relationship_object_id': 'id'})
    
    merge = comps.merge(founders_by_comp, how = 'left', on = 'id')
    
    return merge

## Adding Female Founders 

In [109]:
def n_female_founders(comps, founders, relationships):
    '''get your numbers of females for each company ;) '''
    
    merge_1 = relationships.merge(founders[['person_object_id', 'gender']], how = 'left', on = 'person_object_id')

    merge_1 = merge_1[merge_1.founder ==True]
    merge_1 ['n_female_foundes'] = merge_1.gender.map(lambda x: 1 if x=='Female' else 0)
    merge_1 = pd.DataFrame(merge_1.groupby('relationship_object_id')['n_female_foundes'].sum()).reset_index()
    merge_1 = merge_1.rename(columns = {'relationship_object_id':'id'})
    merge_2 = comps.merge(merge_1, how = 'left', on = 'id')

    return merge_2

,relationship_object_id,Female_bin
0,c:1,0
1,c:10,0
2,c:100,0
3,c:10002,0
4,c:100062,0
5,c:1001,0
6,c:10010,0
7,c:10011,0
8,c:10012,0
9,c:10014,0


In [73]:
relationships

,relationship_id,person_object_id,relationship_object_id,start_at,end_at,is_past,title,founder
0,1,p:2,c:1,NaN,NaN,0,co-founder/ceo/board of directors,True
1,2,p:3,c:1,NaN,NaN,1,vp marketing,False
2,3,p:4,c:3,NaN,NaN,0,evangelist,False
3,4,p:5,c:3,2006-03-01,2009-12-01,1,senior director strategic alliances,False
4,6,p:7,c:4,2005-07-01,2010-04-05,1,chief executive officer,False
5,7,p:8,c:4,NaN,NaN,1,senior software engineer,False
6,8,p:9,c:4,NaN,NaN,1,systems engineering manager,False
7,9,p:10,c:5,NaN,NaN,0,"founder and ceo, board of directors",True
8,10,p:11,c:5,NaN,NaN,1,co-founder,True
9,11,p:12,c:5,NaN,NaN,1,"chief revenue officer, vp of operations",False
